In [29]:
import pandas as pd
import numpy as np 
import pywt
from scipy.stats import moment, skew, kurtosis
from scipy.fft import fft
from scipy import signal
import pandas as pd
import numpy as np
from scipy.stats import moment, skew, kurtosis
from scipy.fft import fft
from scipy import signal
import pywt
from statsmodels.tsa.stattools import acf, pacf
import scipy.stats as stats
import matplotlib.pyplot as plt



In [2]:
dataset_dir = "dataset.xlsx"
dataset = pd.read_excel(dataset_dir)
label, dataset = dataset["label"], dataset.drop(columns=["label"])

In [3]:
for index, row in dataset.iterrows():
    print(f"Index: {index}")
    print(f"Row: {row}")
    break

Index: 0
Row: I_A    0.682026
I_B    0.382088
I_C   -1.069550
Name: 0, dtype: float64


In [17]:

# Compute lag features across the whole dataframe 
lags = [1]
for lag in lags:
    dataset[f'I_A_lag_{lag}'] = dataset['I_A'].shift(lag)
    dataset[f'I_B_lag_{lag}'] = dataset['I_B'].shift(lag)
    dataset[f'I_C_lag_{lag}'] = dataset['I_C'].shift(lag)

# Compute rolling window features across the whole dataframe
dataset['I_A_rolling_mean'] = dataset['I_A'].rolling(window=3).mean()
dataset['I_A_rolling_std'] = dataset['I_A'].rolling(window=3).std()


# Define a feature extraction function
def extract_features(row):
    features = {}
    row_values = row.values
    
    # Higher-order statistics
    features['mean_absolute_deviation'] = np.mean(np.abs(row_values - np.mean(row_values)))
    features['interquartile_range'] = np.percentile(row_values, 75) - np.percentile(row_values, 25)
    
    # Fourier Transform Coefficients
    fft_coeffs = np.abs(fft(row_values))
    features['fft_coefficient_1'] = fft_coeffs[0]
    features['fft_coefficient_2'] = fft_coeffs[1]
    
    # Wavelet Transform Coefficients
    coeffs, _ = pywt.dwt(row_values, 'db1')
    features['wavelet_coefficient_1'] = coeffs[0]
    features['wavelet_coefficient_2'] = coeffs[1]
    
    # Polynomial Features
    features['I_A^3'] = row['I_A']**3
    features['I_B^3'] = row['I_B']**3
    features['I_C^3'] = row['I_C']**3
    features['I_A^2 * I_B'] = (row['I_A']**2) * row['I_B']
    
    # Rolling Window Features - Precomputed
    features['I_A_rolling_mean'] = row['I_A_rolling_mean']
    features['I_A_rolling_std'] = row['I_A_rolling_std']
    
    # Energy and Power Features
    features['instantaneous_power'] = row['I_A']**2 + row['I_B']**2 + row['I_C']**2
    
    # Statistical Moments
    features['third_moment'] = moment(row_values, moment=3)
    features['fourth_moment'] = moment(row_values, moment=4)
    
    # Derivative Features
    features['first_derivative_I_A'] = np.diff([row['I_A']])[0] if len([row['I_A']]) > 1 else np.nan
    features['second_derivative_I_A'] = np.diff([row['I_A']], n=2)[0] if len([row['I_A']]) > 2 else np.nan
    features['first_derivative_I_B'] = np.diff([row['I_B']])[0] if len([row['I_B']]) > 1 else np.nan
    features['second_derivative_I_B'] = np.diff([row['I_B']], n=2)[0] if len([row['I_B']]) > 2 else np.nan
    features['first_derivative_I_C'] = np.diff([row['I_C']])[0] if len([row['I_C']]) > 1 else np.nan
    features['second_derivative_I_C'] = np.diff([row['I_C']], n=2)[0] if len([row['I_C']]) > 2 else np.nan
    
    # Autocorrelation and Partial Autocorrelation
    if len([row['I_A']]) > 1:
        features['autocorr_I_A'] = acf([row['I_A']], nlags=min(10, len([row['I_A']])//2))[1] if len([row['I_A']]) > 1 else np.nan
        features['pacf_I_A'] = pacf([row['I_A']], nlags=min(10, len([row['I_A']])//2))[1] if len([row['I_A']]) > 1 else np.nan
    else:
        features['autocorr_I_A'] = np.nan
        features['pacf_I_A'] = np.nan

    if len([row['I_B']]) > 1:
        features['autocorr_I_B'] = acf([row['I_B']], nlags=min(10, len([row['I_B']])//2))[1] if len([row['I_B']]) > 1 else np.nan
        features['pacf_I_B'] = pacf([row['I_B']], nlags=min(10, len([row['I_B']])//2))[1] if len([row['I_B']]) > 1 else np.nan
    else:
        features['autocorr_I_B'] = np.nan
        features['pacf_I_B'] = np.nan

    if len([row['I_C']]) > 1:
        features['autocorr_I_C'] = acf([row['I_C']], nlags=min(10, len([row['I_C']])//2))[1] if len([row['I_C']]) > 1 else np.nan
        features['pacf_I_C'] = pacf([row['I_C']], nlags=min(10, len([row['I_C']])//2))[1] if len([row['I_C']]) > 1 else np.nan
    else:
        features['autocorr_I_C'] = np.nan
        features['pacf_I_C'] = np.nan
    
    return pd.Series(features)

# Iterate over the dataset to compute features
total_rows = len(dataset)
for index, row in dataset.iterrows():
    dataset.loc[index, "mean"] = row.mean()
    dataset.loc[index, "median"] = row.median()
    dataset.loc[index, "std"] = row.std()
    dataset.loc[index, "var"] = row.var()
    dataset.loc[index, "min"] = row.min()
    dataset.loc[index, "max"] = row.max()
    dataset.loc[index, "range"] = (row.max()-row.min())
    dataset.loc[index, "skew"] = row.skew()
    dataset.loc[index, "kurtosis"] = row.kurtosis()
    dataset.loc[index, "mode"] = row.mode()[0]  # mode returns a Series, take the first element
    dataset.loc[index, "autocorrelation"] = row.autocorr()
    dataset.loc[index, "sum"] = row.sum()
    dataset.loc[index, "i_a-i_b"] = (row['I_A'] - row['I_B'])
    dataset.loc[index, "i_b-i_c"] = (row['I_B'] - row['I_C'])
    dataset.loc[index, "i_a-i_c"] = (row['I_A'] - row['I_C'])
    dataset.loc[index, "i_a*i_b"] = (row['I_A'] * row['I_B'])
    dataset.loc[index, "i_b*i_c"] = (row['I_B'] * row['I_C'])
    dataset.loc[index, "i_a*i_c"] = (row['I_A'] * row['I_C'])
    dataset.loc[index, "i_a*i_b*i_c"] = (row['I_A'] * row['I_B'] * row['I_C'])
    dataset.loc[index, "i_a/i_b"] = (row['I_A'] / row['I_B'])
    dataset.loc[index, "i_b/i_c"] = (row['I_B'] / row['I_C'])
    dataset.loc[index, "i_a/i_c"] = (row['I_A'] / row['I_C'])
    dataset.loc[index, "i_a^2"] = (row['I_A'] * row['I_A'])
    dataset.loc[index, "i_b^2"] = (row['I_B'] * row['I_B'])
    dataset.loc[index, "i_c^2"] = (row['I_C'] * row['I_C'])

    # Extract additional features
    additional_features = extract_features(row)
    for feature, value in additional_features.items():
        dataset.loc[index, feature] = value

    # Print progress
    if (index + 1) % 100 == 0 or (index + 1) == total_rows:
        print(f"Processed {index + 1} out of {total_rows} rows ({(index + 1) / total_rows:.2%})")

print("Feature extraction complete.")

Processed 100 out of 479232 rows (0.02%)
Processed 200 out of 479232 rows (0.04%)
Processed 300 out of 479232 rows (0.06%)
Processed 400 out of 479232 rows (0.08%)
Processed 500 out of 479232 rows (0.10%)
Processed 600 out of 479232 rows (0.13%)
Processed 700 out of 479232 rows (0.15%)
Processed 800 out of 479232 rows (0.17%)
Processed 900 out of 479232 rows (0.19%)
Processed 1000 out of 479232 rows (0.21%)
Processed 1100 out of 479232 rows (0.23%)
Processed 1200 out of 479232 rows (0.25%)
Processed 1300 out of 479232 rows (0.27%)
Processed 1400 out of 479232 rows (0.29%)
Processed 1500 out of 479232 rows (0.31%)
Processed 1600 out of 479232 rows (0.33%)
Processed 1700 out of 479232 rows (0.35%)
Processed 1800 out of 479232 rows (0.38%)
Processed 1900 out of 479232 rows (0.40%)
Processed 2000 out of 479232 rows (0.42%)
Processed 2100 out of 479232 rows (0.44%)
Processed 2200 out of 479232 rows (0.46%)
Processed 2300 out of 479232 rows (0.48%)
Processed 2400 out of 479232 rows (0.50%)
P

In [18]:
dataset.to_csv("enhanced_dataset.csv", index=False)

In [24]:
# Iterate over columns to process NaN values
for column in dataset.columns:
    nan_count = dataset[column].isna().sum()
    
    # Print the number of NaNs for each column
    print(f"Column '{column}' has {nan_count} NaN values.")
    
    if 0 <= nan_count <= 99:
        mean_value = dataset[column].mean()
        dataset[column].fillna(mean_value, inplace=True)
    elif nan_count > 99:
        dataset.drop(columns=[column], inplace=True)


Column 'I_A' has 0 NaN values.
Column 'I_B' has 0 NaN values.
Column 'I_C' has 0 NaN values.
Column 'mean' has 0 NaN values.
Column 'median' has 0 NaN values.
Column 'std' has 0 NaN values.
Column 'var' has 0 NaN values.
Column 'min' has 0 NaN values.
Column 'max' has 0 NaN values.
Column 'range' has 0 NaN values.
Column 'skew' has 0 NaN values.
Column 'kurtosis' has 0 NaN values.
Column 'mode' has 0 NaN values.
Column 'autocorrelation' has 0 NaN values.
Column 'sum' has 0 NaN values.
Column 'i_a-i_b' has 0 NaN values.
Column 'i_b-i_c' has 0 NaN values.
Column 'i_a-i_c' has 0 NaN values.
Column 'i_a*i_b' has 0 NaN values.
Column 'i_b*i_c' has 0 NaN values.
Column 'i_a*i_c' has 0 NaN values.
Column 'i_a*i_b*i_c' has 0 NaN values.
Column 'i_a/i_b' has 0 NaN values.
Column 'i_b/i_c' has 0 NaN values.
Column 'i_a/i_c' has 0 NaN values.
Column 'i_a^2' has 0 NaN values.
Column 'i_b^2' has 0 NaN values.
Column 'i_c^2' has 0 NaN values.
Column 'I_A_lag_1' has 0 NaN values.
Column 'I_B_lag_1' h

In [26]:
dataset.to_csv("enhanced_dataset.csv", index=False)

In [33]:

# Function to detect outliers using the IQR method
def detect_outliers_iqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (data < lower_bound) | (data > upper_bound)

# Function to detect outliers using the Z-score method
def detect_outliers_zscore(data, threshold=3):
    z_scores = np.abs(stats.zscore(data))
    return z_scores > threshold

# Function to combine both methods
def detect_strong_outliers(data):
    outliers_iqr = detect_outliers_iqr(data)
    outliers_zscore = detect_outliers_zscore(data)
    return outliers_iqr & outliers_zscore

# Function to check if the data is normally distributed
def is_normal(data, alpha=0.05):
    k2, p = stats.normaltest(data.dropna())
    return p > alpha

# Iterate over columns to detect and handle outliers
for column in dataset.columns:
    if dataset[column].dtype in ['float64', 'int64']:
        strong_outliers = detect_strong_outliers(dataset[column])
        outlier_count = strong_outliers.sum()
        
        print(f"Column '{column}' has {outlier_count} strong outliers (both IQR and Z-score methods).")
        
        if outlier_count > 0:
            if is_normal(dataset[column]):
                mean_value = dataset[column].mean()
                dataset.loc[strong_outliers, column] = mean_value
                print(f"Outliers in column '{column}' replaced with mean value {mean_value}.")
            else:
                median_value = dataset[column].median()
                dataset.loc[strong_outliers, column] = median_value
                print(f"Outliers in column '{column}' replaced with median value {median_value}.")


Column 'I_A' has 0 strong outliers (both IQR and Z-score methods).
Column 'I_B' has 0 strong outliers (both IQR and Z-score methods).
Column 'I_C' has 0 strong outliers (both IQR and Z-score methods).
Column 'mean' has 1 strong outliers (both IQR and Z-score methods).
Outliers in column 'mean' replaced with median value 0.00308967593373437.
Column 'median' has 0 strong outliers (both IQR and Z-score methods).
Column 'std' has 1 strong outliers (both IQR and Z-score methods).
Outliers in column 'std' replaced with median value 0.7535243492952781.
Column 'var' has 1 strong outliers (both IQR and Z-score methods).
Outliers in column 'var' replaced with median value 0.5677989449808727.
Column 'min' has 1 strong outliers (both IQR and Z-score methods).
Outliers in column 'min' replaced with median value -0.925614.
Column 'max' has 1 strong outliers (both IQR and Z-score methods).
Outliers in column 'max' replaced with median value 0.9248945.
Column 'range' has 1 strong outliers (both IQR an

In [34]:
dataset.to_csv("enhanced_dataset.csv", index=False)